In [2]:
import pandas as pd
geolocation=pd.read_csv("olist_geolocation_dataset.csv")
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [3]:
pd.value_counts(geolocation["geolocation_state"])

SP    404268
MG    126336
RJ    121169
RS     61851
PR     57859
SC     38328
BA     36045
GO     20139
ES     16748
PE     16432
DF     12986
MT     12031
CE     11674
PA     10853
MS     10431
MA      7853
PB      5538
RN      5041
PI      4549
AL      4183
TO      3576
SE      3563
RO      3478
AM      2432
AC      1301
AP       853
RR       646
Name: geolocation_state, dtype: int64

In [4]:
geolocation[geolocation["geolocation_city"]=="sao paulo"].iloc[0]

geolocation_zip_code_prefix         1037
geolocation_lat                 -23.5456
geolocation_lng                 -46.6393
geolocation_city               sao paulo
geolocation_state                     SP
Name: 0, dtype: object

In [5]:
geolocation[geolocation["geolocation_city"]=="são paulo"].iloc[1]

geolocation_zip_code_prefix         1037
geolocation_lat                 -23.5452
geolocation_lng                 -46.6379
geolocation_city               são paulo
geolocation_state                     SP
Name: 14, dtype: object

In [6]:
mapping=pd.DataFrame(geolocation.groupby("geolocation_zip_code_prefix")["geolocation_city"].unique())
mapping["len"]=mapping["geolocation_city"].apply(lambda x:len(x))
mapping=mapping[mapping["len"]>1]
mapping.head()

,geolocation_city,len
geolocation_zip_code_prefix,,
1001,"[sao paulo, são paulo]",2
1002,"[sao paulo, são paulo]",2
1003,"[sao paulo, são paulo]",2
1004,"[sao paulo, são paulo]",2
1005,"[sao paulo, são paulo]",2


In [7]:
list(mapping[mapping["len"]>3].iloc[0][0])

['embu-guacu', 'embu-guaçu', 'embu guaçu', 'embu guacu', 'embuguacu']

In [8]:
geo=pd.DataFrame(geolocation.groupby("geolocation_zip_code_prefix")[["geolocation_lat","geolocation_lng"]].median())
print(geo.shape)
geo

(19015, 2)


,geolocation_lat,geolocation_lng
geolocation_zip_code_prefix,,
1001,-23.550381,-46.634027
1002,-23.548551,-46.635072
1003,-23.548977,-46.635313
1004,-23.549535,-46.634771
1005,-23.549612,-46.636532
...,...,...
99960,-27.953797,-52.029641
99965,-28.179542,-52.035551
99970,-28.343273,-51.873734


In [9]:
import geocoder
import numpy as np
#import time
#data=[]
def get_city(x):
    g=geocoder.osm(x, method='reverse')
    if g.json!=None:
        data=g.json
        if "city" in data.keys():
            city=data.get('city', '')
        elif "town" in data.keys():
            city=data.get('town', '')
        else :
            city=data.get('village', '')
        return city
    else:
        return np.nan
#for i in range(len(geo)):
#    try:
#        x=[geo["geolocation_lat"].iloc[i],geo["geolocation_lng"].iloc[i]]
#        data.append(get_city(x))
#        print(i)
#   except:
#        time.sleep(10)
#       x=[geo["geolocation_lat"].iloc[i],geo["geolocation_lng"].iloc[i]]
#        data.append(get_city(x))
#       print(i)

In [10]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True
#print(isEnglish("são paulo"))
#print(isEnglish("sao paulo"))
for i in ['embu-guacu', 'embu-guaçu', 'embu guaçu', 'embu guacu', 'embuguacu']:
    print(isEnglish(i))

True
False
False
True
True


In [11]:
import re 
print(re.sub('[^A-Za-z0-9 ]+', ' ',"embu guacu" ))
print(re.sub('[^A-Za-z0-9 ]+', ' ',"embu-guacu" ))

embu guacu
embu guacu


In [12]:
import unicodedata
city = " são paulo"
normalized = unicodedata.normalize('NFD', city)
new_city = u"".join([c for c in normalized if not unicodedata.combining(c)])
print(new_city.strip())
def clean_word(x):
    normalized = unicodedata.normalize('NFD', x)
    new_city = u"".join([c for c in normalized if not unicodedata.combining(c)])
    return new_city.strip()

sao paulo


In [13]:
geolocation["geolocation_city"]=geolocation["geolocation_city"].apply(lambda x:re.sub('[^A-Za-z0-9 ]+', '', x ) if isEnglish(x) else re.sub('[^A-Za-z0-9 ]+', '', clean_word(x)))

In [14]:
mapping=pd.DataFrame(geolocation.groupby("geolocation_zip_code_prefix")["geolocation_city"].unique())
mapping["len"]=mapping["geolocation_city"].apply(lambda x:len(x))
mapping[mapping["len"]>1]

,geolocation_city,len
geolocation_zip_code_prefix,,
1307,"[sao paulo, sao bernardo do campo]",2
3203,"[sao paulo, jundiai]",2
4004,"[sao paulo, taboao da serra]",2
4132,"[sao paulo, saopaulo]",2
4346,"[sao paulo, sp]",2
...,...,...
95181,"[farroupilha, jansen]",2
95272,"[otavio rocha, flores da cunha]",2
96222,"[rio grande, quinta]",2


In [15]:
print(geolocation[~geolocation["geolocation_zip_code_prefix"].isin(mapping[mapping["len"]>1].index)].groupby("geolocation_zip_code_prefix")[["geolocation_lat","geolocation_lng"]].median().shape)
is_clean=geolocation[~geolocation["geolocation_zip_code_prefix"].isin(mapping[mapping["len"]>1].index)].groupby(["geolocation_zip_code_prefix","geolocation_city","geolocation_state"])[["geolocation_lat","geolocation_lng"]].median()

(18473, 2)


In [16]:
need_clean=geolocation[geolocation["geolocation_zip_code_prefix"].isin(mapping[mapping["len"]>1].index)].groupby(["geolocation_zip_code_prefix","geolocation_state"])[["geolocation_lat","geolocation_lng"]].median()
print(need_clean.shape)
need_clean.head()

(545, 2)


,,geolocation_lat,geolocation_lng
geolocation_zip_code_prefix,geolocation_state,,
1307,SP,-23.552836,-46.653169
3203,SP,-23.601068,-46.567908
4004,SP,-23.573635,-46.645831
4132,SP,-23.613091,-46.617707
4346,SP,-23.640462,-46.652122


In [17]:
city=[]
for i in range(len(need_clean)):
    x=[need_clean["geolocation_lat"].iloc[i],need_clean["geolocation_lng"].iloc[i]]
    city.append(get_city(x))

In [18]:
print(len(city))
need_clean["geolocation_city"]=city
need_clean=need_clean.dropna()
need_clean["geolocation_city"]=need_clean["geolocation_city"].apply(lambda x:clean_word(x).lower() if isEnglish(x)==False  else x.lower())
need_clean=need_clean.reset_index()
need_clean=need_clean[["geolocation_zip_code_prefix","geolocation_city","geolocation_state","geolocation_lat","geolocation_lng"]]
need_clean.head()

545


,geolocation_zip_code_prefix,geolocation_city,geolocation_state,geolocation_lat,geolocation_lng
0,1307,sao paulo,SP,-23.552836,-46.653169
1,3203,sao paulo,SP,-23.601068,-46.567908
2,4004,sao paulo,SP,-23.573635,-46.645831
3,4132,sao paulo,SP,-23.613091,-46.617707
4,4346,sao paulo,SP,-23.640462,-46.652122


In [23]:
geo_final=pd.concat([is_clean.reset_index(),need_clean],axis=0)

geo_final.sample(20)

,geolocation_zip_code_prefix,geolocation_city,geolocation_state,geolocation_lat,geolocation_lng
8893,36773,cataguases,MG,-21.381722,-42.703802
1544,3820,sao paulo,SP,-23.495852,-46.502477
10628,48903,juazeiro,BA,-9.413363,-40.505063
6367,21320,rio de janeiro,RJ,-22.894145,-43.347175
13546,68738,santa maria do para,PA,-1.347212,-47.574143
10028,44042,feira de santana,BA,-12.215407,-38.996516
539,2331,sao paulo,SP,-23.482953,-46.613530
2504,5103,sao paulo,SP,-23.514833,-46.752853
15378,78553,sinop,MT,-11.823278,-55.515857
11703,58406,campina grande,PB,-7.215811,-35.870841
